In [2]:
import numpy as np
import CoolProp as CP
import pandas as pd

## Compressible Equations

The compressible equations are given by:
$$ (Cf^{''})^{'} + ff^{''} =0$$
$$ \left(\frac{C}{Pr}g_e^{'}\right)^{'} + fg_e^{'} + C\frac{u_e}{h_e}\left(f^{''}\right)^2 =0$$

With the boundary conditions:
$$f(0)=0,\quad f^{'}(0)=0,\quad g_e(0)=g_{e-w}$$

## Derivation

Define the following variables:
$$
\begin{align}
A &= Cf^{''} \\
g &= f^{'}  \\
h &= g^{'}  \\
N &= \frac{C}{Pr}g_e^{'}
\end{align}
$$

Pluggin into original equations gives:
$$
\begin{align}
A' + fh &= 0 \\
f' &= g \\
g' &= h \\
N^{'} + f\frac{Pr}{C}N + C \frac{u_e}{h_e}t^2 &= 0
\end{align}
$$


This gives the following ODEs to be solved

$$
\begin{align}
A' &= - ft  \\
f' &= g \\
g' &= h \\
N^{'} &= - f\frac{Pr}{C}N - C \frac{u_e}{h_e}t^2 \\
g_e^{'} &= \frac{Pr(g_e)}{C(g_e)}N \\
\end{align} 
$$

With:
$$
\begin{align}
Pr(g_e) &= \mu c_p(g_e) k(g_e) \\
C(g_e) &=\frac{\rho(g_e) \mu(g_e)}{\rho_e\rho_e}
\end{align} 
$$